**Document Chains Demo**
- Stuff
- Refine
- Map Reduce

In [1]:
import os
import getpass
import textwrap

from langchain import PromptTemplate, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter


from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
import os

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.5, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))

**Stuff Chain**

In [22]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("resume_sample.pdf")
docs = loader.load()


In [23]:
cnt = 0
for doc in docs:
    cnt += 1
    print(f"Document {cnt}:")
    print(doc.page_content)
    print("\n\n")

Document 1:
ALEX MORGAN
AI Engineer
Contact:alex.morgan@email.com | (555) 123-4567 | linkedin.com/in/alexmorgan
Location: San Francisco, CA
Portfolio: github.com/alexmorgan | alexmorgan.dev
PROFESSIONAL SUMMARY
Logical and detail-oriented AI Engineer with 5 years of experience designing and implementing
production-grade machine learning systems. Specialized in natural language processing,
reinforcement learning, and computer vision with a proven track record of reducing inference
times by 40% and improving model accuracy by 25%. Experienced in full ML lifecycle from
research and prototyping to deployment and monitoring.
SKILLS
Programming Languages: Python, C++, JavaScript, SQL
ML Frameworks: PyTorch, TensorFlow, Keras, Scikit-learn, Hugging Face Transformers
Cloud & MLOps: AWS (SageMaker, Lambda), Docker, Kubernetes, MLflow, Kubeflow, Airflow
Data Processing: Pandas, NumPy, Dask, Apache Spark, PostgreSQL
NLP & Computer Vision: BERT, GPT, YOLO, ResNet, Transformers, OpenCV
Software Eng

In [24]:
prompt_template = """
You are given a Resume as the below text.
------
{text}
------
Question: Please respond with the Key Skills and Experience summary of the person.
Key Skills:
Experience Summary:
"""

In [25]:
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

output_summary = stuff_chain.run(docs)

In [28]:
print(output_summary)

**Key Skills:**

*   **Programming Languages:** Python, C++, JavaScript, SQL
*   **ML Frameworks:** PyTorch, TensorFlow, Keras, Scikit-learn, Hugging Face Transformers
*   **Cloud & MLOps:** AWS (SageMaker, Lambda), Docker, Kubernetes, MLflow, Kubeflow, Airflow
*   **Data Processing:** Pandas, NumPy, Dask, Apache Spark, PostgreSQL
*   **NLP & Computer Vision:** BERT, GPT, YOLO, ResNet, Transformers, OpenCV
*   **Software Engineering:** Git, CI/CD, Agile/Scrum, Test-Driven Development

**Experience Summary:**

Alex Morgan is an AI Engineer with 5 years of experience in designing, implementing, and deploying production-grade machine learning systems. Their expertise spans natural language processing, reinforcement learning, and computer vision. They have a proven track record of improving model performance and efficiency, including reducing inference times and increasing model accuracy. Alex has experience leading teams, architecting distributed training pipelines, and implementing monit

**Refine Chain**


In [29]:
refine_chain = load_summarize_chain(llm, chain_type="refine")
print(refine_chain.refine_llm_chain.prompt.template)

Your job is to produce a final summary.
We have provided an existing summary up to a certain point: {existing_answer}
We have the opportunity to refine the existing summary (only if needed) with some more context below.
------------
{text}
------------
Given the new context, refine the original summary.
If the context isn't useful, return the original summary.


In [31]:
output_summary = refine_chain.run(docs)
print(output_summary)

Alex Morgan is a San Francisco-based AI Engineer with 5 years of experience in designing and implementing production-grade machine learning systems, specializing in NLP, reinforcement learning, and computer vision. Proven ability to improve model performance (e.g., 40% inference time reduction) and lead teams, as demonstrated by leading a team at TechVision AI where they improved user engagement by 22% and reduced LLM training time by 35%. Proficient in a wide range of programming languages, ML frameworks, cloud technologies, and data processing tools. Experience includes designing monitoring systems for model drift (reducing manual intervention by 80%), implementing A/B testing frameworks, and developing sentiment analysis systems processing 10M+ reviews daily with 92% accuracy. Further experience includes implementing deep reinforcement learning algorithms for autonomous navigation (achieving 95% success in simulated urban environments), developing U-Net based architectures for medic

Alex Morgan is a San Francisco-based AI Engineer with 5 years of experience in designing and implementing production-grade machine learning systems, specializing in NLP, reinforcement learning, and computer vision. Proven ability to improve model performance (e.g., 40% inference time reduction) and lead teams, as demonstrated by leading a team at TechVision AI where they improved user engagement by 22% and reduced LLM training time by 35%. Proficient in a wide range of programming languages, ML frameworks, cloud technologies, and data processing tools. Experience includes designing monitoring systems for model drift (reducing manual intervention by 80%), implementing A/B testing frameworks, and developing sentiment analysis systems processing 10M+ reviews daily with 92% accuracy. Further experience includes implementing deep reinforcement learning algorithms for autonomous navigation (achieving 95% success in simulated urban environments), developing U-Net based architectures for medical image segmentation (improving accuracy by 18% over previous methods), and creating a transformer-based multilingual question-answering system supporting 8 languages and handling 1000+ queries per minute. Holds a Master of Science in Computer Science from Stanford University with a specialization in AI and a Bachelor of Science in Mathematics from UC Berkeley. Certified as an AWS Certified Machine Learning - Specialty (2023), Google Professional Machine Learning Engineer (2022), and completed the Deep Learning Specialization on Coursera (2021). Author of publications at NeurIPS (2023) and ICML (2022) focusing on efficient fine-tuning strategies for LLMs and optimizing transformer models for edge devices. Fluent in English, with intermediate Mandarin Chinese and basic Spanish proficiency.

**MAP-Reduce Chain**

In [32]:
map_reduce_chain = load_summarize_chain(llm, chain_type="map_reduce",verbose=True)


In [34]:
print(map_reduce_chain.llm_chain.prompt.template)

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [35]:
output_summary = map_reduce_chain.run(docs)
print(output_summary)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"ALEX MORGAN
AI Engineer
Contact:alex.morgan@email.com | (555) 123-4567 | linkedin.com/in/alexmorgan
Location: San Francisco, CA
Portfolio: github.com/alexmorgan | alexmorgan.dev
PROFESSIONAL SUMMARY
Logical and detail-oriented AI Engineer with 5 years of experience designing and implementing
production-grade machine learning systems. Specialized in natural language processing,
reinforcement learning, and computer vision with a proven track record of reducing inference
times by 40% and improving model accuracy by 25%. Experienced in full ML lifecycle from
research and prototyping to deployment and monitoring.
SKILLS
Programming Languages: Python, C++, JavaScript, SQL
ML Frameworks: PyTorch, TensorFlow, Keras, Scikit-learn, Hugging Face Transformers
Cloud & MLOps: AWS (SageMaker, Lambda), Docker, Kubernetes, MLflow, Kubeflow, Airflow
D